In [1]:
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity

In [2]:
scaler=MinMaxScaler()
#content similarity
content_features=pd.read_csv('./data/content_features.csv',index_col='anime_id')
f_df=pd.read_csv("./data/temp.csv",index_col=0)
content_df=pd.read_csv('./data/content_based_data.csv',index_col='anime_id')

content_df.fillna("unknown",inplace=True)
cf_index=pd.read_csv('./data/cf_index.csv',index_col=0)
pop_anime=list(cf_index['anime_id'])

cv=TfidfVectorizer(stop_words='english')
vectors=cv.fit_transform(content_features['tags']).toarray()

categorical=f_df.drop(['anime_id', 'url', 'title','episodes','aired', 'duration', 'score', 'scored_by', 'rank',
       'popularity', 'synopsis', 'year','genres', 'themes','image_url', 'small_image_url', 'large_image_url',
       'embed_url', 'yt_image'],axis=1)
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
onehot_encoded = onehot_encoder.fit_transform(categorical)
weight_tfidf = 0.85
weight_onehot = 0.15
combined_features = hstack([weight_tfidf*vectors, weight_onehot*onehot_encoded])
cosine_sim_combined = linear_kernel(combined_features, combined_features)
cosine_sim_combined=scaler.fit_transform(cosine_sim_combined)

# collaborative similarity
item_factors = np.loadtxt('./data/items.txt', dtype=float)
cf_similarity=cosine_similarity(item_factors)
cf_similarity=scaler.fit_transform(cf_similarity)

In [6]:
cosine_sim_combined

array([[1.        , 0.46401908, 0.2133589 , ..., 0.14461369, 0.23134134,
        0.12110306],
       [0.47740334, 1.        , 0.19946137, ..., 0.13828609, 0.22668007,
        0.15732515],
       [0.24346644, 0.21038287, 1.        , ..., 0.21358544, 0.20906669,
        0.11930875],
       ...,
       [0.14810113, 0.1198199 , 0.18562258, ..., 1.        , 0.16359089,
        0.22699741],
       [0.23003012, 0.20552157, 0.17618725, ..., 0.15873422, 1.        ,
        0.09888035],
       [0.13997779, 0.15430367, 0.1039261 , ..., 0.2405015 , 0.11973392,
        1.        ]])

In [8]:
comp=cosine_sim_combined.astype(np.float16)

In [9]:
npz_file = 'content_sim_compressed.npz'
np.savez_compressed(npz_file, array=comp)

In [15]:
comp_cf=cf_similarity.astype(np.float16)
npz_file = 'cf_sim_compressed.npz'
np.savez_compressed(npz_file, array=comp_cf)

In [13]:
# import numpy as np
import joblib

# Create a 6029x6029 array with random data and convert to float16
# array = np.random.rand(6029, 6029).astype(np.float16)

# Save the array using joblib with compression
joblib_file = 'con1_compressed.joblib'
joblib.dump(comp, joblib_file, compress=9)  # Higher values for more compression (3 is a good balance)

print(f"Array saved to {joblib_file}")

Array saved to con1_compressed.joblib


In [14]:
# npz_file = 'array_compressed.npz'
loaded_data = np.load(npz_file)

# Access the array using the key you used to save it (in this case, 'array')
loaded_array = loaded_data['array']
loaded_array

array([[1.     , 0.464  , 0.2134 , ..., 0.1447 , 0.2313 , 0.1211 ],
       [0.4773 , 1.     , 0.1995 , ..., 0.1383 , 0.2267 , 0.1573 ],
       [0.2434 , 0.2103 , 1.     , ..., 0.2136 , 0.2091 , 0.1193 ],
       ...,
       [0.1481 , 0.1198 , 0.1857 , ..., 1.     , 0.1636 , 0.227  ],
       [0.23   , 0.2056 , 0.1761 , ..., 0.1587 , 1.     , 0.0989 ],
       [0.14   , 0.1543 , 0.10394, ..., 0.2405 , 0.11975, 1.     ]],
      dtype=float16)

In [16]:
loaded_data = np.load(npz_file)

# Access the array using the key you used to save it (in this case, 'array')
loaded_array = loaded_data['array']
loaded_array

array([[1.     , 0.3564 , 0.1954 , ..., 0.1009 , 0.0976 , 0.07794],
       [0.3564 , 1.     , 0.1324 , ..., 0.1249 , 0.08734, 0.11884],
       [0.179  , 0.11456, 1.     , ..., 0.1012 , 0.1083 , 0.0991 ],
       ...,
       [0.06744, 0.0923 , 0.0865 , ..., 1.     , 0.4934 , 0.3901 ],
       [0.0692 , 0.0586 , 0.09875, ..., 0.4963 , 1.     , 0.4978 ],
       [0.0604 , 0.102  , 0.1004 , ..., 0.401  , 0.504  , 1.     ]],
      dtype=float16)